In [1]:
import pandas as pd
import numpy as np
import sklearn.preprocessing
import sklearn.tree
import seaborn as sns
from xgboost import XGBClassifier
import bp_preprocessing as preprocessing

In [2]:
dev2 = pd.read_csv("dev2.csv")
test2 = pd.read_csv("test2.csv")

In [3]:
train, val = preprocessing.split_cats_by_tolerance(dev2,tolerance=0.01,
                                                   silent=False,
                                                   randomstate=98281,
                                                   split=0.15,step=1,
                                                   categories=['Healthy','AD_MCI','PD','PD_MCI_LBD'])

{'Healthy': 112, 'AD_MCI': 37, 'PD_MCI_LBD': 27, 'PD': 26}
{'Healthy': 20, 'AD_MCI': 6, 'PD': 5, 'PD_MCI_LBD': 5}
Randstate: 98450

Percent Healthy in dev, test: [0.5544554455445545, 0.5555555555555556] 
Standard deviation of these values: 0.0005500550055005382 


Percent AD_MCI in dev, test: [0.18316831683168316, 0.16666666666666666] 
Standard deviation of these values: 0.008250825082508254 


Percent PD in dev, test: [0.12871287128712872, 0.1388888888888889] 
Standard deviation of these values: 0.0050880088008800894 


Percent PD_MCI_LBD in dev, test: [0.13366336633663367, 0.1388888888888889] 
Standard deviation of these values: 0.002612761276127612 



In [4]:
train_eq = preprocessing.over_under(train, val)
train_eq.head()

randomstate


,group,index,assay_ID,KV37,LV469,LV861,LVX54,LV746,LV218,LV316,...,EMIL3,ABCD2,TEN1,PCDAD,ITM2B,ADSV,A0A1W2PRN1,APOF,DCBD2,LMF2
0,AD_MCI,12,TPAD0345,27.128300,22.490591,24.366686,14.622427,24.308411,23.808042,29.455989,...,20.683266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AD_MCI,44,TPAD0459,28.140365,22.316123,22.022777,21.095018,25.128743,23.709564,23.372253,...,20.278083,NaN,NaN,17.810185,15.134276,15.916138,NaN,20.809915,NaN,15.715994
2,AD_MCI,20,TPAD0379,28.008380,22.335054,23.780271,19.596119,25.159908,24.208077,22.160258,...,20.826790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AD_MCI,42,TPAD0453,29.275378,23.853495,24.376810,24.110526,27.073875,25.099886,24.022414,...,20.212292,NaN,NaN,18.241763,17.462557,21.379617,NaN,20.985136,NaN,17.114851
4,AD_MCI,35,TPAD0438,29.546093,24.517895,25.299832,24.264938,27.077570,25.419064,24.137406,...,20.040244,NaN,NaN,17.900476,18.053292,17.812961,NaN,20.704142,NaN,17.436361


In [5]:
train_eq = train_eq.drop(columns = ['index', 'assay_ID'])
train_eq['group'].replace(['AD_MCI', 'Healthy', 'PD', 'PD_MCI_LBD'], [0,1,2,3], inplace=True)
val_eq = val.drop(columns = ['index', 'assay_ID'])
val_eq['group'].replace(['AD_MCI', 'Healthy', 'PD', 'PD_MCI_LBD'], [0,1,2,3], inplace=True)

In [6]:
train_cols = train_eq.loc[:, train_eq.columns != 'group']
val_cols = val_eq.loc[:, val_eq.columns != 'group']
train_X = train_cols.values
val_X = val_cols.values
train_y = train_eq['group'].values.reshape(-1,1)
val_y= val_eq['group'].values.reshape(-1,1)

In [7]:
model = XGBClassifier()
model.fit(train_X, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [8]:
y_pred = model.predict(val_X)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = sklearn.metrics.balanced_accuracy_score(val_y, predictions)
print("Accuracy: %.2f%%" % (accuracy*100))

Accuracy: 64.17%
